In [ ]:
# Third-party
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import h5py

from pyia import GaiaData

import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic
import superfreq as sf

In [ ]:
gc_frame = coord.Galactocentric(z_sun=0*u.pc, galcen_distance=8.3*u.kpc)

In [ ]:
# see FGK-select.ipynb
# stype = 'fgk'
stype = 'af'

if stype == 'af':
    vmax = 1E2
    hex_h = 150 # pc
    
elif stype == 'fgk':
    vmax = 3e2
    hex_h = 120

g = GaiaData('../data/{0}.fits'.format(stype))

c = g.skycoord
galcen = c.transform_to(gc_frame)

In [ ]:
gal = c.galactic
gal.set_representation_cls('cartesian')

In [ ]:
v = galcen.velocity.norm()
plt.hist(v.value[np.isfinite(v)], bins=np.linspace(0, 1000, 128));
plt.yscale('log')

In [ ]:
mw = gp.MilkyWayPotential()
H = gp.Hamiltonian(mw)
w0 = gd.PhaseSpacePosition(galcen.cartesian)[108]

In [ ]:
integrate_time = 128 * 250*u.Myr
print(integrate_time.to(u.Gyr))

orbit = mw.integrate_orbit(w0, dt=0.5*u.Myr, t1=0*u.Myr,
                           t2=integrate_time)

In [ ]:
_ = orbit.plot()

In [ ]:
t2s = [2, 4, 8, 16, 32] * u.Gyr
acts = []
for t2 in t2s:
    orbit = mw.integrate_orbit(w0, dt=0.5*u.Myr, t1=0*u.Myr,
                               t2=t2)
    res = gd.find_actions(orbit, 6)
    acts.append(res['actions'].value)
    
acts = np.array(acts)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4), 
                         sharex=True, sharey=True)
for i in range(3):
    axes[i].plot(t2s, np.abs((acts[:, i] - acts[-1, i]) / acts[-1, i]))
    axes[i].axhline(0.001)
    
axes[i].set_xscale('log', basex=2)
axes[i].set_yscale('log')
axes[i].xaxis.set_ticks(2 ** np.arange(1, 6+1))
axes[i].set_ylim(1E-5, 1e0)

In [ ]:
res['freqs']

In [ ]:
def cartesian_to_poincare_polar(w):
    r"""
    Convert an array of 6D Cartesian positions to Poincaré
    symplectic polar coordinates. These are similar to cylindrical
    coordinates.
    Parameters
    ----------
    w : array_like
        Input array of 6D Cartesian phase-space positions. Should have
        shape ``(norbits,6)``.
    Returns
    -------
    new_w : :class:`~numpy.ndarray`
        Points represented in 6D Poincaré polar coordinates.
    """

    R = np.sqrt(w[..., 0]**2 + w[..., 1]**2)
    # phi = np.arctan2(w[...,1], w[...,0])
    phi = np.arctan2(w[..., 0], w[..., 1])

    vR = (w[..., 0]*w[..., 0+3] + w[..., 1]*w[..., 1+3]) / R
    vPhi = w[..., 0]*w[..., 1+3] - w[..., 1]*w[..., 0+3]

    # pg. 437, Papaphillipou & Laskar (1996)
    sqrt_2THETA = np.sqrt(np.abs(2*vPhi))
    pp_phi = sqrt_2THETA * np.cos(phi)
    pp_phidot = sqrt_2THETA * np.sin(phi)

    z = w[..., 2]
    zdot = w[..., 2+3]

    new_w = np.vstack((R.T, pp_phi.T, z.T,
                       vR.T, pp_phidot.T, zdot.T)).T
    return new_w


def worker(task):
    i, w0 = task
    w0 = gd.PhaseSpacePosition.from_w(w0, galactic)

    # logging.debug('Starting orbit {0}'.format(i))

    try:
        orbit = mw.integrate_orbit(w0, dt=0.5*u.Myr, t1=0*u.Myr,
                                  t2=integrate_time)
    except Exception as e:
        print('Failed to integrate orbit {0}\n{1}'
                     .format(i, str(e)))
        return i, np.full(3, np.nan)

    new_ws = cartesian_to_poincare_polar(orbit.w().T).T
    fs = [(new_ws[j] + 1j*new_ws[j+3]) for j in range(3)]

    freq = sf.SuperFreq(orbit.t.value, p=4)
    try:
        res = freq.find_fundamental_frequencies(fs)
    except Exception as e:
        print('Failed to compute frequencies for orbit {0}\n{1}'
                     .format(i, str(e)))
        return i, np.full(3, np.nan)
    
    return i, res.fund_freqs
    
_, freqs = worker((0, w0.w(galactic)))

In [ ]:
freqs

In [ ]:
np.abs(freqs / res['freqs'])

In [ ]:
freqs, res['freqs']

In [ ]:
2*np.pi / freqs, 2*np.pi / res['freqs']

In [ ]:
data = dict()

In [ ]:
res['actions'].to(u.km/u.s*u.kpc)

In [ ]:
orbit.zmax(approximate=True)

orbit.pericenter(approximate=True)

orbit.apocenter(approximate=True)

In [ ]:
Lz = orbit[0].angular_momentum()[2]

In [ ]:
E = orbit.energy()[0]

In [ ]:
x0 = w0.cylindrical.rho.to(u.pc).value
x0_c = [x0, 0, 0] * u.pc
v0 = mw.circular_velocity(x0_c)[0].to(u.km/u.s).value
v0_c = [0, v0, 0] * u.km/u.s
w0_c = gd.PhaseSpacePosition(pos=x0_c, vel=v0_c)
Ec = w0_c.energy(H)[0]

In [ ]:
E_Ec = E - Ec

In [ ]:
E_Ec.to((u.kpc/u.Myr)**2)

In [ ]:
derp = Table.read('../data/R1500_z_750_cyl_rv.fits')

In [ ]:
derp[(derp['parallax'] > 0) & ((derp['parallax']/derp['parallax_error']) > 6)][16:32].write('../data/all-rv.fits', overwrite=True)